# Numeric Table Detection


The objective is to demonstrate detection of tables in raw texxt to use targeted table extraction and lower the cost of using Textract.
Exmple for 1M pages per month containing 19% of pages with at leaqst 1 table. 

https://calculator.aws/#/estimate?id=fb01232c5a8b8304d1c1169b2911af087467ea82



# Notebook Setup

In this step, we will import some necessary libraries that will be used throughout this notebook. 

In [2]:
!python -m pip install -q amazon-textract-response-parser==0.1.44
!python -m pip install -q amazon-textract-caller==0.0.28
!python -m pip install -q amazon-textract-prettyprinter==0.0.16
!python -m pip install -q amazon-textract-textractor==1.1.1
!python -m pip install -q pypdf==3.1.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-textractor 1.1.1 requires tabulate==0.9.*, but you have tabulate 0.8.10 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-prettyprinter 0.0.16 requires tabulate==0.8.10, but you have tabulate 0.9.0 which is incompatible.


In [3]:
import boto3
import botocore
from IPython.display import Image, display, JSON, HTML
from textractcaller.t_call import call_textract, Textract_Features, call_textract_expense
from textractprettyprinter.t_pretty_print import convert_table_to_list
from trp import Document
import os


# variables

s3=boto3.client('s3')
# because the reference document is stored in us-east-2, we run Textract in us-east-2 as well
textract = boto3.client('textract', region_name='us-east-2')


In [4]:
file="./DemoTable.pdf"
s3url="s3://amazon-textract-public-content/code-samples/DemoTable.pdf"

In [5]:
from IPython.display import IFrame
IFrame(file, width=1500, height=600)

---
Extract unstructured data with Amazon Textract 

Amazon Textract is an ML powered OCR service that is capable of detecting and extracting text from documents. Text data in the form of WORDS and LINES can be extracted from documents using Amazon Textract `DetectDocumentText` API. Let's extract the words and lines from the demonstration document.

In [6]:
# Call Amazon Textract
response = call_textract(input_document=s3url, boto3_textract_client=textract)

text=""
# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        text=text+"\n"+item["Text"]
print (text)


Sample
Amazon Textract
Document
Subtitle
Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Maecenas
porttitor congue massa. Fusce posuere, magna sed pulvinar
ultricies, purus lectus malesuada libero, sit amet commodo magna
eros quis urna.
Nunc viverra imperdiet enim. Fusce est. Vivamus a tellus.
Pellentesque habitant morbi tristique senectus et netus et malesuada
fames ac turpis egestas. Proin pharetra nonummy pede. Mauris et
orci.
Aenean nec lorem. In porttitor. Donec laoreet nonummy augue.
Suspendisse dui purus, scelerisque at, vulputate vitae, pretium
mattis, nunc. Mauris eget neque at sem venenatis eleifend. Ut
nonummy.
Heading 1
Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Maecenas porttitor congue massa. Fusce posuere, magna sed
pulvinar ultricies, purus lectus malesuada libero, sit amet commodo magna eros quis urna.
Nunc viverra imperdiet enim. Fusce est. Vivamus a tellus.
Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis 

---
# Detect Tables in raw text

In this example using a threashold of 4 characters out of the 15 most used characters being digits works well to identify if the page is mostly constituted of numeric tables.



In [7]:
from collections import Counter


def containsatable(textcontent):
    counter = Counter(textcontent)
    mostcom = counter.most_common(15)
    countdigit=0
    for i in mostcom :
        if (i[0].isdigit()):
            countdigit=countdigit+1
    return(countdigit>=4)

# Can we provide a sample impl to find tables, even when there is a lot of text around it? 
# Checking if a certain number of lines has a high digit count


In [8]:
from trp import Document
import math
doc = Document(response)
#print(doc.pages[0])
# Iterate over elements in the document
pagenum=0
pageswithtables=[]
for page in doc.pages:
    pagenum=pagenum+1
    text=''
    for line in page.lines:
        text=text+line.text
    if (containsatable(text)):
        pageswithtables.append(pagenum)

print("List of pages with Tables:")        
print (pageswithtables)
print("Percentage of pages with tables:")    
print (str(len(pageswithtables))+"/"+str(pagenum))
print (str(math.trunc(len(pageswithtables)/pagenum*100))+"%")
    


List of pages with Tables:
[5, 13]
Percentage of pages with tables:
2/13
15%


# Call Analyse Documents only on the pages with tables
First, we build a smaller PDF document containing only the pages identified as containing tables and then we run the AnalyzeDocument call on the new document.

In [9]:
from pypdf import PdfReader, PdfWriter

pdf_file_path = file
file_base_name = pdf_file_path.replace('.pdf', '')

pdf = PdfReader(pdf_file_path)

pages = pageswithtables
pdfWriter = PdfWriter()

for page_num in pages:
    pdfWriter.add_page(pdf.pages[page_num-1])

pdf_out = open("temp.pdf", 'wb')
pdfWriter.write(pdf_out)
pdf_out.close()


In [11]:
from IPython.display import IFrame
IFrame('temp.pdf', width=1500, height=600)

In [22]:
!python -m pip install pandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [23]:
# When you are using your own document, you need to copy the result to S3 and process that.
# In our sample case, we already uploaded the output to S3, so the next line is commented out
# !aws s3 cp temp.pdf s3://{data_bucket}/temp.pdf --only-show-errors
s3url="s3://amazon-textract-public-content/code-samples/DemoTableDocument-ExtractedTablePages.pdf"

In [25]:
import pandas as pd
resp = call_textract(input_document=s3url, features=[Textract_Features.TABLES])
tdoc = Document(resp)
dfs = list()

for page in tdoc.pages:
    for table in page.tables:
        tab_list = convert_table_to_list(trp_table=table)
        print(tab_list)
        dfs.append(pd.DataFrame(tab_list))

df1 = dfs[0]
df2 = dfs[1]

[['Year ', 'Product ', 'Cost ', 'Quantity ', 'Total ', 'Sales ', 'Margin '], ['2000 ', 'Demo ', '$ 0.62 ', '1514 ', '$ 938.68 ', '$ 1,228.91 ', '$ 290.23 '], ['2001 ', 'Demo ', '$ 4.16 ', '3610 ', '$15,017.60 ', '$18,164.12 ', '$ 3,146.52 '], ['2002 ', 'Demo ', '$ 6.47 ', '1239 ', '$ 8,016.33 ', '$13,313.82 ', '$ 5,297.49 '], ['2003 ', 'Demo ', '$ 8.97 ', '4457 ', '$39,979.29 ', '$29,973.74 ', '$ (10,005.55) '], ['2004 ', 'Demo ', '$ 8.89 ', '1094 ', '$ 9,725.66 ', '$ 9,155.22 ', '$ (570.44) '], ['2005 ', 'Demo ', '$ 3.50 ', '3246 ', '$11,361.00 ', '$13,419.90 ', '$ 2,058.90 '], ['2006 ', 'Demo ', '$ 9.40 ', '2847 ', '$26,761.80 ', '$22,285.93 ', '$ (4,475.87) '], ['2007 ', 'Demo ', '$ 9.87 ', '4027 ', '$39,746.49 ', '$46,015.01 ', '$ 6,268.52 '], ['2008 ', 'Demo ', '$ 4.27 ', '1116 ', '$ 4,765.32 ', '$ 5,007.16 ', '$ 241.84 '], ['2009 ', 'Demo ', '$ 5.94 ', '3954 ', '$23,486.76 ', '$22,824.95 ', '$ (661.81) '], ['2010 ', 'Demo ', '$ 4.55 ', '3836 ', '$17,453.80 ', '$24,728.83 ', '$ 7,

In [18]:
df1

,0,1,2,3,4,5,6
0,Year,Product,Cost,Quantity,Total,Sales,Margin
1,2000,Demo,$ 0.62,1514,$ 938.68,"$ 1,228.91",$ 290.23
2,2001,Demo,$ 4.16,3610,"$15,017.60","$18,164.12","$ 3,146.52"
3,2002,Demo,$ 6.47,1239,"$ 8,016.33","$13,313.82","$ 5,297.49"
4,2003,Demo,$ 8.97,4457,"$39,979.29","$29,973.74","$ (10,005.55)"
5,2004,Demo,$ 8.89,1094,"$ 9,725.66","$ 9,155.22",$ (570.44)
6,2005,Demo,$ 3.50,3246,"$11,361.00","$13,419.90","$ 2,058.90"
7,2006,Demo,$ 9.40,2847,"$26,761.80","$22,285.93","$ (4,475.87)"
8,2007,Demo,$ 9.87,4027,"$39,746.49","$46,015.01","$ 6,268.52"
9,2008,Demo,$ 4.27,1116,"$ 4,765.32","$ 5,007.16",$ 241.84


In [19]:
df2

,0,1,2,3,4,5,6
0,Year,Product,Cost,Quantity,Total,Sales,Margin
1,2011,Demo,$ 7.52,192,"$ 1,443.84","$ 1,206.10",$ (237.74)
2,2012,Demo,$ 5.18,3569,"$18,487.42","$25,812.35","$ 7,324.93"
3,2013,Demo,$ 5.39,1714,"$ 9,238.46","$ 9,927.26",$ 688.80
4,2014,Demo,$ 1.49,2309,"$ 3,440.41","$ 5,212.94","$ 1,772.53"
5,2015,Demo,$ 0.98,684,$ 670.32,$ 874.55,$ 204.23
6,2016,Demo,$ 8.84,4551,"$40,230.84","$29,368.05","$ (10,862.79)"
7,2017,Demo,$ 2.87,4636,"$13,305.32","$22,201.79","$ 8,896.47"
8,2018,Demo,$ 6.14,3281,"$20,145.34","$16,663.61","$ (3,481.73)"
9,2019,Demo,$ 5.62,4960,"$27,875.20","$34,032.56","$ 6,157.36"
